# Libraries

## a. Install Libraries

In [1]:
!pip install transformers datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [2]:
!apt install git-lfs
!git config --global user.email "wdevinsp@gmail.com"
!git config --global user.name "WilliamDevin23"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer
from transformers.keras_callbacks import PushToHubCallback
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import json
import pandas as pd

## b. Import Pre-Trained Model IndoBERT and the Tokenizer for QnA

In [5]:
url = "indolem/indobert-base-uncased"

In [6]:
indobert_model = TFAutoModelForQuestionAnswering.from_pretrained(url, from_pt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(url)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Prepare Data

## a. Import CSV

In [8]:
df = pd.read_csv("/content/Digestive_Information.csv")

In [9]:
df.head()

,context,question,answers,answer_start
0,Gastritis adalah peradangan pada dinding lambu...,Apa yang dimaksud dengan gastritis?,Gastritis adalah peradangan pada dinding lambu...,0
1,Gastritis adalah peradangan pada dinding lambu...,Apa itu gastritis?,Gastritis adalah peradangan pada dinding lambu...,0
2,Gastritis adalah peradangan pada dinding lambu...,Apa definisi dari gastritis?,Gastritis adalah peradangan pada dinding lambu...,0
3,Gastritis adalah peradangan pada dinding lambu...,Bagaimana penjelasan tentang gastritis?,Gastritis adalah peradangan pada dinding lambu...,0
4,Gastritis adalah peradangan pada dinding lambu...,Apa yang dimaksud dengan peradangan lambung (g...,Gastritis adalah peradangan pada dinding lambu...,0


## b. Change to JSON File

In [10]:
def pd_to_squad_format(df):
  qa_arr = []
  for record in df.values :
    qa_dict = {'question':'',
               'context':'',
               'answers': {'text': [], 'answer_start': []}}
    qa_dict['question'] = record[1]
    qa_dict['context'] = record[0]
    qa_dict['answers']['text'].append(record[2])
    qa_dict['answers']['answer_start'].append(record[3])
    qa_arr.append(qa_dict)
  return qa_arr

In [11]:
qa_arr = pd_to_squad_format(df)

In [12]:
qa_arr[0]

{'question': 'Apa yang dimaksud dengan gastritis?',
 'context': 'Gastritis adalah peradangan pada dinding lambung yang ditandai dengan nyeri di ulu hati atau lambung. Jika dibiarkan, gastritis bisa berlangsung bertahun-tahun dan menyebabkan komplikasi serius, seperti tukak lambung. Gastritis terbagi dua, yaitu gastritis akut dan kronis. Gastritis akut terjadi ketika peradangan di lapisan lambung berlangsung secara tiba-tiba. Kondisi ini menyebabkan nyeri ulu hati hebat yang bersifat sementara. Namun, jika tidak ditangani, gastritis akut bisa berlanjut menjadi kronis. Sementara pada gastritis kronis, peradangan di lapisan lambung terjadi secara bertahap. Nyeri akibat gastritis kronis lebih ringan daripada gastritis akut, tetapi terjadi lebih sering dan lebih lama. Dinding lambung berfungsi menghasilkan enzim pencernaan dan asam lambung, serta memproduksi lendir untuk melindungi lapisan lambung dari kerusakan akibat asam lambung. Gastritis terjadi ketika dinding lambung mengalami peradan

In [13]:
with open('digestive_dataset.json', 'w') as qa_json:
  json.dump(qa_arr, qa_json)

## c. Load into Dataset Format

In [14]:
base_dir = "/content/"

In [15]:
dataset = load_dataset("json", data_files=base_dir+"digestive_dataset.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
dataset['train'][1]

{'question': 'Apa itu gastritis?',
 'context': 'Gastritis adalah peradangan pada dinding lambung yang ditandai dengan nyeri di ulu hati atau lambung. Jika dibiarkan, gastritis bisa berlangsung bertahun-tahun dan menyebabkan komplikasi serius, seperti tukak lambung. Gastritis terbagi dua, yaitu gastritis akut dan kronis. Gastritis akut terjadi ketika peradangan di lapisan lambung berlangsung secara tiba-tiba. Kondisi ini menyebabkan nyeri ulu hati hebat yang bersifat sementara. Namun, jika tidak ditangani, gastritis akut bisa berlanjut menjadi kronis. Sementara pada gastritis kronis, peradangan di lapisan lambung terjadi secara bertahap. Nyeri akibat gastritis kronis lebih ringan daripada gastritis akut, tetapi terjadi lebih sering dan lebih lama. Dinding lambung berfungsi menghasilkan enzim pencernaan dan asam lambung, serta memproduksi lendir untuk melindungi lapisan lambung dari kerusakan akibat asam lambung. Gastritis terjadi ketika dinding lambung mengalami peradangan. Tergantung p

# Preprocess Function

In [20]:
#Global variables
MAX_LENGTH = 512
STRIDE = 256

In [21]:
def preprocess_dataset(dataset):
  tokenized_data = tokenizer(dataset['question'],
                             dataset['context'],
                             max_length=MAX_LENGTH,
                             truncation='only_second',
                             return_overflowing_tokens=True,
                             return_offsets_mapping=True,
                             stride=STRIDE,
                             padding="max_length")
  sample_map = tokenized_data.pop('overflow_to_sample_mapping')
  offset_map = tokenized_data.pop('offset_mapping')

  tokenized_data['start_positions'] = []
  tokenized_data['end_positions'] = []

  for i, offset in enumerate(offset_map):
    input_ids = tokenized_data['input_ids'][i]
    cls_token_idx = input_ids.index(tokenizer.cls_token_id)
    sequence_id = tokenized_data.sequence_ids(i)
    sample_idx = sample_map[i]
    answer = dataset['answers'][sample_idx]
    if len(answer['answer_start']) == 0 :
      tokenized_data['start_positions'].append(cls_token_idx)
      tokenized_data['end_positions'].append(cls_token_idx)
    else :
      start = answer['answer_start'][0]
      end = start + len(answer['text'][0])

    answer_start_idx = 0
    while sequence_id[answer_start_idx] != 1 :
      answer_start_idx += 1

    answer_end_idx = len(input_ids) - 1
    while sequence_id[answer_end_idx] != 1 :
      answer_end_idx -= 1

    if (offset[answer_start_idx][0] <= start and offset[answer_end_idx][1] >= end) :
      while (answer_start_idx < len(offset) and offset[answer_start_idx][0] <= start) :
        answer_start_idx += 1
      tokenized_data['start_positions'].append(answer_start_idx - 1)

      while (offset[answer_end_idx][1] >= end) :
        answer_end_idx -= 1
      tokenized_data['end_positions'].append(answer_end_idx + 1)

    else :
      tokenized_data['start_positions'].append(cls_token_idx)
      tokenized_data['end_positions'].append(cls_token_idx)
  return tokenized_data

In [22]:
data = dataset.map(preprocess_dataset, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

In [23]:
data['train'][5]['end_positions']

25

In [24]:
data['train'][5]['start_positions']

9

In [25]:
tokenizer.decode(data['train'][5]['input_ids'][9:26])

'gastritis adalah peradangan pada dinding lambung yang ditandai dengan nyeri di ulu hati atau lambung'

# Fine-Tune the Model

In [26]:
model_base_name = url.split('/')[-1]
push_name = f'{model_base_name}-finetuned-digestive-qna'
learning_rate = 2e-5
epochs = 50
weight_decay = 0.01

In [27]:
train_set = indobert_model.prepare_tf_dataset(data['train'],
                                     shuffle=True)

In [28]:
from transformers import create_optimizer

train_steps = len(train_set) * epochs

optimizer, _ = create_optimizer(init_lr=learning_rate, num_warmup_steps=0,
                                num_train_steps=train_steps)

In [29]:
indobert_model.compile(optimizer=optimizer, jit_compile=True, metrics=['accuracy'])

In [30]:
class myCallback(tf.keras.callbacks.Callback) :
  def on_epoch_end(self, epoch, logs=None) :
    if logs.get('start_logits_accuracy') >= 0.96 and logs.get('end_logits_accuracy') >= 0.96 :
      self.model.stop_training = True

In [31]:
mycallback = myCallback()

In [32]:
push_to_hub_callback = PushToHubCallback(
    output_dir="./qa_model_gerd_anxiety",
    tokenizer=tokenizer,
    hub_model_id=push_name,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/wdevinsp/indobert-base-uncased-finetuned-digestive-qna into local empty directory.


In [33]:
# Fit the model
indobert_model.fit(train_set, epochs=epochs, callbacks=[mycallback, push_to_hub_callback])

Epoch 1/50


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 6/74 [=>............................] - ETA: 54s - loss: 5.8541 - end_logits_accuracy: 0.0208 - start_logits_accuracy: 0.0417

74/74 [==============================] - 153s 951ms/step - loss: 3.2094 - end_logits_accuracy: 0.3834 - start_logits_accuracy: 0.3598
Epoch 2/50
74/74 [==============================] - 103s 1s/step - loss: 1.9506 - end_logits_accuracy: 0.4240 - start_logits_accuracy: 0.4358
Epoch 3/50
74/74 [==============================] - 111s 2s/step - loss: 1.6907 - end_logits_accuracy: 0.4358 - start_logits_accuracy: 0.6081
Epoch 4/50
74/74 [==============================] - 112s 2s/step - loss: 3.2723 - end_logits_accuracy: 0.1875 - start_logits_accuracy: 0.3007
Epoch 5/50
74/74 [==============================] - 105s 1s/step - loss: 3.4151 - end_logits_accuracy: 0.1875 - start_logits_accuracy: 0.2551
Epoch 6/50
74/74 [==============================] - 113s 2s/step - loss: 2.1764 - end_logits_accuracy: 0.2686 - start_logits_accuracy: 0.4358
Epoch 7/50
74/74 [==============================] - 106s 1s/step - loss: 1.4706 - end_logits_accuracy: 0.4105 - start_logits_accuracy: 0.5473
Epoch 8/50
74/

# Inference

## a. GERD

In [46]:
context = r"""GERD adalah penyakit yang terjadi ketika asam lambung naik ke kerongkongan.
Gejala GERD bisa berupa sensasi perih di dada dan perut, rasa pahit di mulut, nyeri ulu hati,
mual, dan sulit menelan. Otot bagian bawah kerongkongan atau lower esophageal sphincter (LES)
normalnya akan terbuka saat menelan, kemudian menutup setelah makanan turun ke lambung. Namun,
otot ini dapat melemah, misalnya akibat penyakit atau pola hidup yang kurang sehat. Otot LES yang
tidak bisa menutup sepenuhnya dapat menyebabkan asam lambung naik ke kerongkongan. Kondisi inilah
yang menimbulkan gejala GERD (gastroesophageal reflux disease) atau penyakit asam lambung. Penyebab
GERD adalah melemahnya otot LES sehingga tidak mampu menahan isi lambung atau asam lambung agar tidak
naik ke kerongkongan. Jika terjadi terus-menerus, kondisi ini dapat mengiritasi lapisan kerongkongan
hingga menyebabkan peradangan. Ada beberapa hal yang bisa menjadi penyebab GERD, yaitu obesitas,
kehamilan, usia lanjut, kebiasaan sering berbaring atau tidur setelah makan, gastroparesis, yaitu
melemahnya otot dinding lambung sehingga pengosongan lambung melambat, gangguan jaringan ikat, misalnya
skleroderma atau lupus, penyakit bawaan lahir, seperti hernia hiatus dan atresia esofagus, pernah menjalani
operasi di area dada atau perut bagian atas sehingga melukai kerongkongan, dan efek samping obat-obatan
tertentu, misalnya aspirin, ibuprofen, benzodiazepin, antidepresan, atau obat terapi hormon untuk menopause.
Ada beberapa faktor yang dapat memperparah gejala GERD, seperti kebiasaan merokok atau sering terpapar asap rokok
(perokok pasif), diet ekstrim atau telat makan saat puasa, sering makan dalam porsi besar atau makan pada tengah
malam, mengonsumsi makanan yang asam, berlemak, atau berbumbu pedas, mengonsumsi minuman berkafein, beralkohol,
atau bersoda, mengalami gangguan kecemasan atau stres yang tidak terkelola dengan baik (GERD anxiety)."""

question = "Apa itu GERD?"

In [47]:
inputs = tokenizer(question, context, return_tensors="np")

In [48]:
outputs = indobert_model(inputs)

In [49]:
start_position = np.argmax(outputs.start_logits[0])
end_position = np.argmax(outputs.end_logits[0])
print(start_position)
print(end_position)

# Extract this substring from the inputs
answer = inputs["input_ids"][0, start_position: end_position + 1]
print(tokenizer.decode(answer))

7
20
gerd adalah penyakit yang terjadi ketika asam lambung naik ke kerongkongan


## b. Example from Dataset

In [38]:
context = df['context'].values[0]
question = df['question'].values[0]

In [39]:
inputs = tokenizer(question, context, return_tensors="np")
outputs = indobert_model(inputs)

In [40]:
start_position = np.argmax(outputs.start_logits[0])
end_position = np.argmax(outputs.end_logits[0])

In [41]:
answer = inputs["input_ids"][0, start_position: end_position + 1]
print(tokenizer.decode(answer))

gastritis adalah peradangan pada dinding lambung yang ditandai dengan nyeri di ulu hati atau lambung


## c. Unseen Data

In [42]:
context = """Kanker lambung adalah kanker yang terjadi akibat pertumbuhan sel lambung yang tidak normal dan tidak terkendali. Pertumbuhan sel abnormal ini terjadi karena sel di lambung mengalami perubahan genetik. Pada stadium awal, kanker lambung jarang menimbulkan gejala spesifik. Gejalanya dapat berupa perut kembung atau nyeri ulu hati, dan sering kali hanya dianggap sebagai sakit maag atau menyerupai gejala penyakit lambung pada umumnya. Kondisi tersebut membuat kanker lambung sulit untuk dideteksi lebih awal dan umumnya baru terdiagnosis setelah masuk stadium akhir. Hal ini tentunya akan memengaruhi peluang kesembuhan. Kanker lambung terjadi akibat adanya perubahan (mutasi) genetik pada sel-sel di lambung. Mutasi tersebut menyebabkan sel-sel tersebut tumbuh secara tidak normal dan tidak terkendali sehingga menjadi sel kanker. Penyebab terjadinya perubahan genetik pada sel lambung tersebut belum diketahui dengan pasti. Namun, ada beberapa faktor yang diketahui bisa meningkatkan risiko seseorang untuk mengalami kanker lambung, yaitu merokok, berusia 55 tahun ke atas, berjenis kelamin laki-laki, memiliki anggota keluarga dengan riwayat kanker lambung, dan pernah menjalani operasi pada lambung. Kanker lambung juga lebih berisiko terjadi pada orang yang pernah mengalami penyakit seperti infeksi bakteri pylori, infeksi virus Epstein-Barr (EBV), radang lambung kronis, polip di dalam lambung, anemia akibat kekurangan vitamin B12, daya tahan tubuh lemah, dan menderita kanker jenis lain, seperti limfoma, kanker esofagus, kanker usus, kanker prostat, dan kanker serviks."""
question = "Apa saja faktor risiko terkena kanker lambung?"

In [43]:
inputs = tokenizer(question, context, return_tensors="np")
outputs = indobert_model(inputs)

In [44]:
start_position = np.argmax(outputs.start_logits[0])
end_position = np.argmax(outputs.end_logits[0])

In [45]:
answer = inputs["input_ids"][0, start_position: end_position + 1]
print(tokenizer.decode(answer))

kanker lambung juga lebih berisiko terjadi pada orang yang pernah mengalami penyakit seperti infeksi bakteri pylori, infeksi virus epstein - barr ( ebv ), radang lambung kronis, polip di dalam lambung, anemia akibat kekurangan vitamin b12, daya tahan tubuh lemah, dan menderita kanker jenis lain, seperti limfoma, kanker esofagus, kanker usus, kanker prostat, dan kanker serviks


# Get Pushed Model

In [50]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering

# You can, of course, use your own username and model name here
# once you've pushed your model using the code above!
checkpoint = "wdevinsp/indobert-base-uncased-finetuned-digestive-qna"
model = TFAutoModelForQuestionAnswering.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/440M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at wdevinsp/indobert-base-uncased-finetuned-digestive-qna.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/737k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]